In [9]:
import numpy as np

In [10]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

In [11]:
c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

# SciPy (scipy.optimize.linprog)

In [12]:
from scipy.optimize import linprog
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

# CVXPY

In [62]:
# !pip install cvxpy==1.5.1
# !pip install cvxopt
# !pip install pyscipopt

In [84]:
import cvxpy
import numpy as np

In [50]:
x = cvxpy.Variable(shape=n, integer = True)

In [51]:
A = A.flatten() # Преобразуем размерность массива
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
total_value = cvxpy.sum(cvxpy.multiply(x, c))

In [52]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [53]:
problem.solve()
# бесконечность

-inf

В таком случае будем рассматривать только положительные значения $x$:
$$x \geq 0$$
В переформулированном виде задача будет решаться следующим образом:

### добавляем условие
* x_positive = x >= 0
* constraints=[constraint, x_positive]

In [54]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([0., 0., 0., 7., 0., 0.])

А что если мы можем брать не любое количество товаров, а только один или не брать их вовсе? Задаём $x$ типа boolean.
$$x= 0 \ или \ x = 1$$
Программное решение такой задачи имеет следующий вид:

In [55]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([1., 0., 0., 1., 0., 1.])

#

In [61]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([1., 0., 0., 1., 0., 1.])

# PuLP

In [63]:
# !pip install pulp

In [60]:
from pulp import *

In [64]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B 
#Ограничения
problem += 4*A + 5*B <= 30 
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А: ", A.varValue)
print("Количество автомобилей модели В: ", B.varValue)
print("Суммарный доход: ", value(problem.objective))
#Количество автомобилей модели А:  1.0
#Количество автомобилей модели В:  4.0
#Суммарный доход:  200000.0

Количество автомобилей модели А:  1.0
Количество автомобилей модели В:  4.0
Суммарный доход:  200000.0


c:\Users\Temnykov\AppData\Local\Programs\Python\Python38\lib\site-packages\pulp\pulp.py:1423: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Задание 6.1
Составьте оптимальный план перевозок со склада № 1 и склада № 2 в три торговых центра с учётом тарифов, запасов на складах и потребностей торговых центров, которые указаны в таблице:

| | ТЦ1 (110 шт.) | ТЦ2 (150 шт.) | ТЦ3 (140 шт.) |
|---:|:---:|:---:|:---:|
| Склад №1 (180 шт.) | 2у.е. | 5у.е. | 3у.е. |
| Склад №2 (220 шт.) | 7у.е. | 7у.е. | 6у.е. |

Сформулируйте предложенную задачу как задачу линейного программирования и решите её любым способом (желательно программным).

В качестве ответа введите минимальную суммарную стоимость поставки. Ответ округлите до целого числа.

тоесть
1. есть товар на складах №1 = 180 шт и №2 = 220 шт и эти склады торговым центрам продают товар по разным ценам
2. а торговым центрам нужно только товара ТЦ1 (110 шт.), ТЦ2 (150 шт.), ТЦ3 (140 шт.)
3. и нужно найти минимум

Склад №1 продаст ТЦ1 (110 шт.) по 2у.е. и ТЦ3 (70 шт.) по 3у.е
$$110*2+70*3=220+210=430$$
Склад №2 продаст ТЦ2 (150 шт.) по 7у.е. и ТЦ3 (70 шт.) по 6у.е
$$150*7+70*6=1050+420=1470$$
а в сего
$$430+1470=1900$$

In [103]:
from scipy.optimize import linprog
import numpy as np

In [104]:
cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3

$x_{ij}$ - сколько забирается со i склада клиенту j  
$$f = \sum_{i,j} cost_{ij} * x_{ij}$$

In [105]:
c = cost.flatten()
print(c) 

[2 5 3 7 7 6]


Для каждого склада количество взятых предметов должно быть меньше, чем на складе:

$$\forall i: \sum_j x_{ij} \leq stock_i$$

In [106]:
A = []
b = []
for i in range(0, num_warehouse):
    A.append([0] * (num_clients * i) + [1] * num_clients + [0] * (num_clients * (num_warehouse - i - 1)))
    b.append(stock[i])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]]
[180 220]


Для каждого клиента количество приобретаемых товаров должно быть больше на единицу, чем спрос:

$$\forall j: \sum_i x_{ij} \geq demand_j$$

Который также:

$$\forall j: - \sum_i x_{ij} \leq -demand_j$$

In [107]:
A = A.tolist()
b = b.tolist()
for j in range(0, num_clients):
    A.append(([0] * j + [-1] + [0] * (num_clients - j - 1)) * num_warehouse)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[ 1  1  1  0  0  0]
 [ 0  0  0  1  1  1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]]
[ 180  220 -110 -150 -140]


In [108]:
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 1900.0
              x: [ 1.100e+02  0.000e+00  7.000e+01  0.000e+00  1.500e+02
                   7.000e+01]
            nit: 5
          lower:  residual: [ 1.100e+02  0.000e+00  7.000e+01  0.000e+00
                              1.500e+02  7.000e+01]
                 marginals: [ 0.000e+00  1.000e+00  0.000e+00  2.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
                 marginals: [-3.000e+00 -0.000e+00 -5.00

Ответ: 110 единиц со склада 1 клиенту 1, 70 единиц со склада 1 клиенту 3,
150 наименований со склада 2 клиенту 2, 70 наименований со склада 2 клиенту 3

### Задание 6.2
В прошлом юните мы обсуждали задачу о назначениях исполнителей задач - теперь пришло время решить её.

Напомним суть: необходимо распределить пять задач между пятью исполнителями таким образом, чтобы суммарные затраты на работы были наименьшими.

![alt text](https://lms-cdn.skillfactory.ru/assets/courseware/v1/3c57d0c14951762fe62a494509d15a69/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md6_6_2.png)

In [95]:
from scipy.optimize import linprog
import numpy as np
import cvxpy as cvx

#### Матрица стоимостей $C$

In [96]:
c = np.array([[1000, 12, 10, 19, 8],
    [12, 1000, 3, 7, 2], 
    [10, 3, 1000, 6, 20], 
    [19, 7, 6, 1000, 4], 
    [8, 2, 20, 4, 1000]])

#### Матрица переменных $X$

In [97]:
x = cvx.Variable(shape=(5,5), boolean=True)

#### Ограничения (сумма $X$ по строкам и столбцам равна 1)

In [98]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5)
]

#### Целевая функция (сумма $CX$)

In [99]:
func = cvx.sum(cvx.multiply(x, c))

In [100]:
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

#### Ответ

In [101]:
problem.solve()

32.0

#### Выбранные ячейки матрицы

In [102]:
np.round(x.value)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

### Задание 6.3
Найдите кратчайший маршрут из точки $A$, который проходит через все другие точки по одному разу и возвращается в $A$.

![alt text](https://i.ibb.co/qJzt4LW/image.png)

In [85]:
import cvxpy as cvx

Пусть X будет массивом булевых значений 5x5. Если X[A, B] = True, то маршрут проходит по ребру (A, B).

In [86]:
x = cvx.Variable(shape=(5, 5), boolean=True)

Пусть C будет массивом целочисленных значений, в котором хранятся расстояния между вершинами.

In [87]:
c = np.array(
    [
        [0, 12, 10, 19, 8],
        [12, 0, 3, 7, 2],
        [10, 3, 0, 6, 20],
        [19, 7, 6, 0, 4],
        [8, 2, 20, 4, 0],
    ]
)

Зададим ограничения. Нетрудно заметить, что кратчайший маршрут будет соединять ровно 5 пар точек – то есть, сумма x по обеим осям будет равна 5.

In [88]:
constraints = [
    cvx.sum(x, axis=0) == np.ones(5),
    cvx.sum(x, axis=1) == np.ones(5),
]

Также заметим, что "нулевое ребро" между каждой точкой и ней самой же не может входить в наш маршрут – то есть, на главной диагонали X располагаются нули.

In [89]:
for i in range(5):
    constraints.append(x[i, i] == 0)

Для исключения коротких циклов A-B-A, из-за которых решение становится несвязным, введём ограничение: путь между двумя точками может быть пройден не более одного раза.

In [90]:
for i in range(4):
    for j in range(i + 1, 5):
        constraints.append(x[i, j] + x[j, i] <= 1)

Определим функцию, считающую длину маршрута.

In [91]:
func = cvx.sum(cvx.multiply(x, c))

Решим задачу.

In [92]:
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)

### Ответ

In [93]:
problem.solve()

32.0

Длина наикратчайшего пути – 32.

### Найденный путь
Посмотрим на найденный наикратчайший путь. Для этого взглянем на значения матрицы X, при которых достигается минимальная длина пути.

In [94]:
np.round(x.value)

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

Это означает, что вершина A - первая в маршруте, B - третья, C - вторая, D - четвёртая. Эту же информацию можно достать из матрицы $X$.  
Искомое решение.

![alt text](https://i.ibb.co/qJzt4LW/image.png)

Обратите внимание, что в заданиях 6.2 и 6.3 для записи двух разных задач используется почти одна и та же система уравнений!